# Running the original DCA implementation

Running experiments with the original DCA had to be on its own notebook because the 

In [9]:
# downgrade may be needed because DCA uses outdated API + versions not specified as requirements to dca package
!pip install --force-reinstall dca tensorflow==1.15 keras==2.3.1 scanpy==1.4.4 scipy==1.3.0

     |████████████████████████████████| 412.3MB 36kB/s 
     |████████████████████████████████| 378kB 45.9MB/s 
     |████████████████████████████████| 1.9MB 46.0MB/s 
     |████████████████████████████████| 25.2MB 1.4MB/s 
     |████████████████████████████████| 14.5MB 154kB/s 
     |████████████████████████████████| 4.0MB 40.0MB/s 
     |████████████████████████████████| 22.2MB 1.4MB/s 
     |████████████████████████████████| 9.5MB 45.7MB/s 
     |████████████████████████████████| 133kB 49.7MB/s 
     |████████████████████████████████| 4.0MB 32.7MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 512kB 45.0MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 3.8MB 38.2MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 1.0MB 39.6MB/s 
     |████████████████████████████████| 276kB 44.4MB/s 


In [12]:
exit() # forces session to restart. Alternatively, can also restart the session manually above.

In [5]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
import os

path = 'drive/My Drive/Denoising sc-RNA-seq/' #add path of project folder in your G-drive
os.chdir(path)

In [4]:
# get sim. data
import pickle 

with open('data/simraw2.pickle', 'rb') as handle:
    simraw2 = pickle.load(handle)

In [5]:
# get assignment data
import preprocess as p

genes = p.get_genes_data('data/gene.csv')

In [ ]:
from dca.api import dca

dca_raw = dca(simraw2, threads=1, copy=True, log1p=False, return_info=True, verbose=False)

In [7]:
dca_raw_norm = p.normalize(dca_raw)

/usr/local/lib/python3.6/dist-packages/scanpy/preprocessing/_simple.py:284: DeprecationWarning: Use is_view instead of isview, isview will be removed in the future.
  if isinstance(data, AnnData) and data.isview:


In [ ]:
dca_genes = dca(genes, threads=1, copy=True, log1p=False, return_info=True, verbose=False)
dca_genes_norm = p.normalize(dca_genes)

In [9]:
with open('data/dca-simraw2.pickle', 'wb') as handle:
    pickle.dump(dca_raw_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/dca-genes.pickle', 'wb') as handle:
    pickle.dump(dca_genes_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
!ls

data		     new_report.ipynb	 __pycache__
final_report.ipynb   original_dca.ipynb  Readme.md
generate_data.ipynb  preprocess.py	 report_template.ipynb
